In [6]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re

In [7]:
df=pd.read_excel('1.xlsx')

In [8]:
df.head(5)

,Tran Date,Value Date,Unnamed: 2,Tran Particulars,Type,Sub Type,Instr,Instr No,Debit,Credit,Balance,Unnamed: 11
0,2022-11-15,2022-11-15,NaN,IFN/FBLEPIFIbXA2xdScupcEednu45HV xxxxxxxx0111,T,CI,NaN,NaN,NaN,100.0,100.0,Cr
1,2022-11-15,2022-11-15,NaN,UPIOUT/231907746601/rafeekrahim25102000@oksb/0000,T,CI,NaN,NaN,99.0,NaN,1.0,Cr
2,2022-11-15,2022-11-16,NaN,UPI IN/232018997547/navarasparayil@okhdfcban/0000,T,CI,NaN,NaN,NaN,200.0,201.0,Cr
3,2022-11-18,2022-11-18,NaN,UPIOUT/232215138698/rafipattazhi123@oksbi/UP/0000,T,CI,NaN,NaN,160.0,NaN,41.0,Cr
4,2022-11-18,2022-11-18,NaN,IFN/FBLEPIFIAUxAsBF8qm8VzgxzNVvd xxxxxxxx0111,T,CI,NaN,NaN,NaN,1010.0,1051.0,Cr


In [9]:

  df.columns

Index(['Tran Date', 'Value Date', 'Unnamed: 2', 'Tran Particulars', 'Type',
       'Sub Type', 'Instr ', 'Instr No', 'Debit', 'Credit', 'Balance',
       'Unnamed: 11'],
      dtype='object')

In [10]:
#Column pre-processing
df=df.drop('Unnamed: 2',axis=True)
df.rename(columns = {'Unnamed: 11':'Place_Value'}, inplace = True)

#Splitting Tran Particulars
split_columns = df['Tran Particulars'].str.split('/', expand=True)
split_columns.columns = [str(i) for i in range(1, len(split_columns.columns) + 1)]
df = pd.concat([df, split_columns], axis=1)
#re-ordering columns
df = df[['Tran Date', 'Value Date', 'Tran Particulars','1',
       '2', '3', '4', '5', 'Type', 'Sub Type',
       'Instr ', 'Instr No', 'Debit', 'Credit', 'Balance', 'Place_Value', ]]
df.rename(columns = {'1':'Particulars_1','2':'Particulars_2','3':'Particulars_3','4':'Particulars_4','5':'Particulars_5'}, inplace = True)
df= df.sort_values(by='Particulars_1')


In [11]:
df.head()

,Tran Date,Value Date,Tran Particulars,Particulars_1,Particulars_2,Particulars_3,Particulars_4,Particulars_5,Type,Sub Type,Instr,Instr No,Debit,Credit,Balance,Place_Value
319,2023-08-30,2023-08-30,FEDERAL BANK:STIPEND FOR AUG 2023,FEDERAL BANK:STIPEND FOR AUG 2023,None,None,None,None,T,BI,NaN,NaN,NaN,25000.00,25572.00,Cr
339,2023-09-13,2023-09-13,FEDERAL BANK:STIPEND FOR SEP 23,FEDERAL BANK:STIPEND FOR SEP 23,None,None,None,None,T,BI,NaN,NaN,NaN,10227.27,25740.93,Cr
174,2023-04-08,2023-04-08,FT IMPS/IFI/309815091771/INSTANTPAY INDIA LIMI...,FT IMPS,IFI,309815091771,INSTANTPAY INDIA LIMITED,,T,CI,NaN,NaN,NaN,1.00,1010.89,Cr
0,2022-11-15,2022-11-15,IFN/FBLEPIFIbXA2xdScupcEednu45HV xxxxxxxx0111,IFN,FBLEPIFIbXA2xdScupcEednu45HV xxxxxxxx0111,None,None,None,T,CI,NaN,NaN,NaN,100.00,100.00,Cr
429,2023-10-06,2023-10-06,IFN/FBLEPIFITajKAi5VWqHkuzN4fYwy xxxxxxxx0111,IFN,FBLEPIFITajKAi5VWqHkuzN4fYwy xxxxxxxx0111,None,None,None,T,CI,NaN,NaN,NaN,414.60,500.00,Cr


In [12]:

condition = df['Particulars_1'].str.contains(r'SBINT:')

df['Particulars_1_Updated'] = np.where(condition, 'SBIINT', df['Particulars_1'])
df['Duration_Intrest_Credited'] = np.where(
    condition,
    df['Particulars_1'].str.extract(r'SBINT:(.*)', expand=False),
    np.nan
)

In [13]:

unique_values = df['Particulars_1_Updated'].unique()
print(unique_values)

['FEDERAL BANK:STIPEND FOR AUG 2023' 'FEDERAL BANK:STIPEND FOR SEP 23'
 'FT IMPS' 'IFN' 'MB FTO' 'POS' 'SBIINT' 'TO ATM' 'TO ECM' 'UPI IN'
 'UPIOUT']


In [20]:
df_transactions = df
df_codes = pd.read_excel('mcc_data.xlsx')

In [21]:
# Ensure that the 'CODE' column in df_codes is of string type for matching
df_codes['CODE'] = df_codes['CODE'].astype(str)

# Function to match the code and return corresponding 'MERCHANT' and 'GROUP_CATEGORY_NEW'
def match_code_merchant_group(row):
    # Check each particulars column for a match in df_codes
    for col in ['Particulars_1', 'Particulars_2', 'Particulars_3', 'Particulars_4', 'Particulars_5']:
        code = row[col]
        if pd.isnull(code):
            continue
        # If the value in the cell is a string with length 4, then it might be a code
        if isinstance(code, str) and len(code) == 4:
            match = df_codes[df_codes['CODE'] == code]
            if not match.empty:
                return match.iloc[0]['CODE_DESC'],match.iloc[0]['MERCHANT'], match.iloc[0]['GROUP_CATEGORY_NEW']
    return None, None, None

# Create new columns for 'MERCHANT' and 'GROUP_CATEGORY_NEW'
df_transactions['CODE_DESC'] = None
df_transactions['MERCHANT'] = None
df_transactions['GROUP_CATEGORY_NEW'] = None

# Apply the function to each row in the dataframe
for index, row in df_transactions.iterrows():
    code_desc,merchant, group_category_new = match_code_merchant_group(row)
    df_transactions.at[index, 'CODE_DESC'] = code_desc
    df_transactions.at[index, 'MERCHANT'] = merchant
    df_transactions.at[index, 'GROUP_CATEGORY_NEW'] = group_category_new



In [22]:
df=df_transactions

In [23]:
grouped_data = df.groupby('Particulars_1_Updated')
dataframes = {name: group for name, group in grouped_data}

In [24]:
df

,Tran Date,Value Date,Tran Particulars,Particulars_1,Particulars_2,Particulars_3,Particulars_4,Particulars_5,Type,Sub Type,...,Instr No,Debit,Credit,Balance,Place_Value,Particulars_1_Updated,Duration_Intrest_Credited,CODE_DESC,MERCHANT,GROUP_CATEGORY_NEW
319,2023-08-30,2023-08-30,FEDERAL BANK:STIPEND FOR AUG 2023,FEDERAL BANK:STIPEND FOR AUG 2023,None,None,None,None,T,BI,...,NaN,NaN,25000.00,25572.00,Cr,FEDERAL BANK:STIPEND FOR AUG 2023,NaN,None,None,None
339,2023-09-13,2023-09-13,FEDERAL BANK:STIPEND FOR SEP 23,FEDERAL BANK:STIPEND FOR SEP 23,None,None,None,None,T,BI,...,NaN,NaN,10227.27,25740.93,Cr,FEDERAL BANK:STIPEND FOR SEP 23,NaN,None,None,None
174,2023-04-08,2023-04-08,FT IMPS/IFI/309815091771/INSTANTPAY INDIA LIMI...,FT IMPS,IFI,309815091771,INSTANTPAY INDIA LIMITED,,T,CI,...,NaN,NaN,1.00,1010.89,Cr,FT IMPS,NaN,None,None,None
0,2022-11-15,2022-11-15,IFN/FBLEPIFIbXA2xdScupcEednu45HV xxxxxxxx0111,IFN,FBLEPIFIbXA2xdScupcEednu45HV xxxxxxxx0111,None,None,None,T,CI,...,NaN,NaN,100.00,100.00,Cr,IFN,NaN,None,None,None
429,2023-10-06,2023-10-06,IFN/FBLEPIFITajKAi5VWqHkuzN4fYwy xxxxxxxx0111,IFN,FBLEPIFITajKAi5VWqHkuzN4fYwy xxxxxxxx0111,None,None,None,T,CI,...,NaN,NaN,414.60,500.00,Cr,IFN,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,2023-04-10,2023-04-10,UPIOUT/310034328944/shahilt038-3@okhdfcbank//0000,UPIOUT,310034328944,shahilt038-3@okhdfcbank,,0000,T,CI,...,NaN,90.0,NaN,88.89,Cr,UPIOUT,NaN,None,None,None
180,2023-04-10,2023-04-10,UPIOUT/310031712724/shamilahammed00@oksbi/UP/0000,UPIOUT,310031712724,shamilahammed00@oksbi,UP,0000,T,CI,...,NaN,200.0,NaN,178.89,Cr,UPIOUT,NaN,None,None,None
179,2023-04-10,2023-04-10,UPIOUT/310020108087/UPICw745E43O/0000,UPIOUT,310020108087,UPICw745E43O,0000,None,T,CI,...,NaN,NaN,70.00,378.89,Cr,UPIOUT,NaN,None,None,None
201,2023-04-23,2023-04-23,UPIOUT/311322998312/rafeekrahim25102000@oksb/0000,UPIOUT,311322998312,rafeekrahim25102000@oksb,0000,None,T,CI,...,NaN,890.0,NaN,3.17,Cr,UPIOUT,NaN,None,None,None


In [25]:
df.to_csv('Expanded_df.csv', index=False)

In [26]:
UPIOUT=dataframes['UPIOUT']

In [27]:
UPIOUT

,Tran Date,Value Date,Tran Particulars,Particulars_1,Particulars_2,Particulars_3,Particulars_4,Particulars_5,Type,Sub Type,...,Instr No,Debit,Credit,Balance,Place_Value,Particulars_1_Updated,Duration_Intrest_Credited,CODE_DESC,MERCHANT,GROUP_CATEGORY_NEW
344,2023-09-15,2023-09-15,UPIOUT/325886923467/q268548705@ybl/UPI/5541,UPIOUT,325886923467,q268548705@ybl,UPI,5541,T,CI,...,NaN,200.0,NaN,21157.93,Cr,UPIOUT,NaN,Service Stations,NaN,Auto and Fuel Payments
321,2023-08-30,2023-08-30,UPIOUT/324240339487/paytmqr28100505010119vp9/5462,UPIOUT,324240339487,paytmqr28100505010119vp9,5462,None,T,CI,...,NaN,195.0,NaN,25197.00,Cr,UPIOUT,NaN,Bakeries,NaN,Food and Beverage
351,2023-09-19,2023-09-19,UPIOUT/362899056903/q097526806@ybl/UPI/5812,UPIOUT,362899056903,q097526806@ybl,UPI,5812,T,CI,...,NaN,523.0,NaN,19329.93,Cr,UPIOUT,NaN,"Eating Places, Restaurants",NaN,Food and Beverage
343,2023-09-15,2023-09-15,UPIOUT/362469989363/hiba1hyder-1@okicici/UPI/0000,UPIOUT,362469989363,hiba1hyder-1@okicici,UPI,0000,T,CI,...,NaN,285.0,NaN,21357.93,Cr,UPIOUT,NaN,None,None,None
345,2023-09-15,2023-09-15,UPIOUT/362466697916/q723840854@ybl/UPI/5812,UPIOUT,362466697916,q723840854@ybl,UPI,5812,T,CI,...,NaN,338.0,NaN,20819.93,Cr,UPIOUT,NaN,"Eating Places, Restaurants",NaN,Food and Beverage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,2023-04-10,2023-04-10,UPIOUT/310034328944/shahilt038-3@okhdfcbank//0000,UPIOUT,310034328944,shahilt038-3@okhdfcbank,,0000,T,CI,...,NaN,90.0,NaN,88.89,Cr,UPIOUT,NaN,None,None,None
180,2023-04-10,2023-04-10,UPIOUT/310031712724/shamilahammed00@oksbi/UP/0000,UPIOUT,310031712724,shamilahammed00@oksbi,UP,0000,T,CI,...,NaN,200.0,NaN,178.89,Cr,UPIOUT,NaN,None,None,None
179,2023-04-10,2023-04-10,UPIOUT/310020108087/UPICw745E43O/0000,UPIOUT,310020108087,UPICw745E43O,0000,None,T,CI,...,NaN,NaN,70.0,378.89,Cr,UPIOUT,NaN,None,None,None
201,2023-04-23,2023-04-23,UPIOUT/311322998312/rafeekrahim25102000@oksb/0000,UPIOUT,311322998312,rafeekrahim25102000@oksb,0000,None,T,CI,...,NaN,890.0,NaN,3.17,Cr,UPIOUT,NaN,None,None,None
